# Running experiments

## Set up

In [ ]:
# data wrangling
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# models
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
import statsmodels.formula.api as smf
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, KFold

# evaluation
from sklearn.metrics import mean_squared_error, r2_score

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
%matplotlib inline

import os
sns.set_context('notebook')
sns.set_style('white')

In [ ]:
# Features for each experiment type 
# (i.e. if experimenting with AOD products, then additional features corresponding to ADD_FEATS['aod_prod'] are included.)
ADD_FEATS = {'none': [], 'aod_only':['AOD'], 'aod_prod':['AOD_absorption', 'AOD_nonspherical', 'small_mode_AOD', \
    'medium_mode_AOD', 'large_mode_AOD'], 'aod_mix':['aod_mix_01', 'aod_mix_02', 'aod_mix_03', 'aod_mix_04', \
    'aod_mix_05', 'aod_mix_06', 'aod_mix_07', 'aod_mix_08', 'aod_mix_09', 'aod_mix_10', 'aod_mix_11', 'aod_mix_12', \
    'aod_mix_13', 'aod_mix_14', 'aod_mix_15', 'aod_mix_16', 'aod_mix_17', 'aod_mix_18', 'aod_mix_19', 'aod_mix_20', \
    'aod_mix_21', 'aod_mix_22', 'aod_mix_23', 'aod_mix_24', 'aod_mix_25', 'aod_mix_26', 'aod_mix_27', 'aod_mix_28', \
    'aod_mix_29', 'aod_mix_30', 'aod_mix_31', 'aod_mix_32', 'aod_mix_33', 'aod_mix_34', 'aod_mix_35', 'aod_mix_36', \
    'aod_mix_37', 'aod_mix_38', 'aod_mix_39', 'aod_mix_40', 'aod_mix_41', 'aod_mix_42', 'aod_mix_43', 'aod_mix_44', \
    'aod_mix_45', 'aod_mix_46', 'aod_mix_47', 'aod_mix_48', 'aod_mix_49', 'aod_mix_50', 'aod_mix_51', 'aod_mix_52', \
    'aod_mix_53', 'aod_mix_54', 'aod_mix_55', 'aod_mix_56', 'aod_mix_57', 'aod_mix_58', 'aod_mix_59', 'aod_mix_60', \
    'aod_mix_61', 'aod_mix_62', 'aod_mix_63', 'aod_mix_64', 'aod_mix_65', 'aod_mix_66', 'aod_mix_67', 'aod_mix_68', \
    'aod_mix_69', 'aod_mix_70', 'aod_mix_71', 'aod_mix_72', 'aod_mix_73', 'aod_mix_74']}
MODEL_FEATS = {}
EXP_NAME = {'none': 'No AOD', 'aod_only': 'AOD Only', 'aod_prod': 'AOD Products', 'aod_mix': 'AOD Mix'}

## Helper functions

In [ ]:
def preproc(df):
    # Create year, month variables
    df['Year'] = pd.DatetimeIndex(df.loc[:, 'Date']).year.astype('category')
    df['Month'] = pd.DatetimeIndex(df.loc[:, 'Date']).month.astype('category')

    # Cast other categorical variables to the right type
    for col in ['POC', 'Site.Code', 'light', 'med', 'heavy', 'ecosys']:
        df[col] = df.loc[:, col].astype('category')

    # Replace NAs in smoke vars with 0
    df['light'].fillna(0, inplace=True)
    df['med'].fillna(0, inplace=True)
    df['heavy'].fillna(0, inplace=True)
    df['ecosys'] = df['ecosys'].cat.add_categories(-1)
    df['ecosys'].fillna(-1, inplace=True)

    # Replace NAs in cluster info vars with 0
    df['frp_avg'].fillna(-999, inplace=True)
    df['frp_vars'].fillna(-999, inplace=True)
    df['num_pts'].fillna(0, inplace=True)

    # First type: create quantiles for fire distance; placeholder "x+1"th quantile for NAs
    df['fire_dist_25'] = pd.qcut(df['fire_dist'], q=[0, .25, .5, .75, 1.], labels=[1,2,3,4])
    df['fire_dist_25'] = df['fire_dist_25'].cat.add_categories(5)
    df['fire_dist_25'].fillna(5, inplace=True)
    
    # Continuous variable that fills NAs of fire_dist with the maximum fire distance + 10
    max_fire = df['fire_dist'].max()
    df['fire_dist_rep-max'] = df['fire_dist'].fillna(max_fire+10)

    return df

In [ ]:
def transform(X, X_test, X_imp=None, y=None, one_hot_cols=[], scaler=StandardScaler(), split=False, exp_type=None):
    """
    Transform X and y (optional) using scaler. If split is True, then the test set is not provided and train_test_split
    is performed. If X_imp is not None, we also transform imputed data using the same scaler.
    
    Returns the transformed X_train, X_test, X_imp, y_train, y_test
    """
    # One-hot code categorical variables
    X_oh = pd.get_dummies(X, columns = one_hot_cols)
    # Save features of exp_type in MODEL_FEATS
    if exp_type is not None:
        MODEL_FEATS[exp_type] = list(X_oh.columns)
    X_arr = np.array(X_oh)
    cont_col = X_oh.select_dtypes(include=[np.float64, np.int64]).columns
    cont_ind = [X_oh.columns.get_loc(c) for c in cont_col]
    if not split:
        # X_test provided explicitly
        X_arr[:,cont_ind] = scaler.fit_transform(X_arr[:,cont_ind])
        X_test_oh = pd.get_dummies(X_test, columns = one_hot_cols)
        X_test_arr = np.array(X_test_oh)
        X_test_arr[:,cont_ind] = scaler.transform(X_test_arr[:,cont_ind])
        X_train_arr, y_train, y_test = X_arr, None, None

    else:
        # split data first
        X_train_arr, X_test_arr, y_train, y_test = train_test_split(X_arr, y, test_size = 0.3, random_state = 88)
        # fit scaler on training data
        X_train_arr[:,cont_ind] = scaler.fit_transform(X_train_arr[:,cont_ind])
        X_test_arr[:,cont_ind] = scaler.transform(X_test_arr[:,cont_ind])
    
    X_imp_arr = None
    
    if X_imp is not None:
        X_imp_oh = pd.get_dummies(X_imp, columns = one_hot_cols)
        X_imp_arr = np.array(X_imp_oh)
        X_imp_arr[:,cont_ind] = scaler.transform(X_imp_arr[:,cont_ind])    

    return X_train_arr, X_test_arr, X_imp_arr, y_train, y_test

### Getting results

In [ ]:
def plot_fitted(true_value, predicted_value, title, save_path):
    plt.figure(figsize=(5,5))
    plt.scatter(true_value, predicted_value, s=5, c="crimson")

    p1 = max(max(predicted_value), max(true_value))
    p2 = min(min(predicted_value), min(true_value))
    plt.plot([p1, p2], [p1, p2], 'b-')
    plt.xlabel('True Values', fontsize=11)
    plt.ylabel('Predictions', fontsize=11)
    plt.axis('equal')

    m, b = np.polyfit(true_value, predicted_value, deg=1)
    plt.axline(xy1=(0, b), slope=m, label=f'$y = {m:.1f}x {b:+.1f}$')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

def get_perf(y_test, preds):
    rmse = round(np.sqrt(mean_squared_error(y_test, preds)), 2)
    r2 = round(r2_score(y_test, preds),2)
    return rmse, r2

def get_perf_df(df, sp, exp_type):
    rmse = round(np.sqrt(mean_squared_error(df['actual'], df['predicted'])), 2)
    r2 = round(r2_score(df['actual'], df['predicted']),2)
    return pd.Series(dict(species=sp, exp_type=exp_type, r2 = r2, rmse = rmse))

def plot_imp(mod, feature_list, title, save_path):
    # Specify feature names (for XGBoost)
    # mod.get_booster().feature_names = feature_list
    # Plot in a graph with top 11 features
    # ax = xgb.plot_importance(mod.get_booster(), max_num_features=10, importance_type='gain')
    # ax.figure.savefig(save_path)
    
    importances = mod.feature_importances_
    sorted_idx = importances.argsort()[::-1][:10][::-1]
    plt.barh(np.array(feature_list)[sorted_idx], importances[sorted_idx])
    plt.xlabel('Relative Importance')
    plt.tight_layout()
    plt.title(title)
    plt.savefig(save_path)
    plt.show()

def create_comp_plot(df, cat, t1, t2):
    #create catplot
    g = sns.catplot(data=df, x='species', y='r2', col='model', hue=cat, kind="bar")

    #move overall title up
    g.fig.subplots_adjust(top=.8)

    #add overall title
    g.set(xlabel="Species", ylabel="r2")
    g.set_titles("{col_name}")
    g.fig.suptitle(t1)
    g.set(ylim=(-1, 1))

    #create catplot
    g = sns.catplot(data=df, x='species', y='rmse', col='model', hue=cat, kind="bar")

    #move overall title up
    g.fig.subplots_adjust(top=.8)

    #add overall title
    g.set(xlabel="Species", ylabel="rmse")
    g.set_titles("{col_name}")
    g.set(ylim=(0, None))
    g.fig.suptitle(t2);

## Read the data

In [ ]:
csn_misr = pd.read_csv('../data/merged/MergedAll_MISR_CSN_2000_2021.csv')
aod_df = preproc(csn_misr.copy())
aod_nonNA = aod_df.dropna(subset = ['AOD'])
aod_NA = aod_df[aod_df['AOD'].isna()]

## Imputation with KNN

In [ ]:
KNN_FEATURES = ['Year', 'Month', 'POC', 'Site.Latitude', 'Site.Longitude', \
    'elevation', 'fire_dist_rep-max', 'light', 'med', 'heavy','frp_avg', 'frp_vars', \
    'num_pts', 'ecosys']

In [ ]:
def impute_knn(df, impute_ft):
    """
    Impute the features impute_ft for dataframe df.

    Returns the imputed copy of the df and indices where features have been imputed.
    """
    new_df = df.copy()
    imp_ind = []
    for var in impute_ft:
        df_nonNA = df.dropna(subset = [var])
        df_NA = df[df[var].isna()]
        imp_ind.extend(list(df_NA.index))
        if df_NA.empty:
            continue
        X = df_nonNA.loc[:,KNN_FEATURES]
        y = df_nonNA.loc[:,'AOD']
        X_test = df_NA.loc[:,KNN_FEATURES]
        print("Non-NAs: {} \nNAs: {}".format(X.shape[0], X_test.shape[0]))
        
        X_train_oh, X_test_oh, _, _, _ = transform(X, X_test, one_hot_cols = ['POC', 'ecosys'])
        knn = KNeighborsRegressor(n_neighbors=10,weights='distance')
        knn.fit(X_train_oh, y)
        preds_knn = knn.predict(X_test_oh)

        print("Training RMSE & R2:", get_perf(y, knn.predict(X_train_oh)))
        print("Total values interpolated [{}] : {}".format(var, preds_knn.shape[0]))
        preds_knn = pd.Series(preds_knn, index = df_NA.index)
        new_df[var].fillna(preds_knn, inplace=True)
    return new_df, imp_ind

In [ ]:
df_imp_aod, aod_imp_ind = impute_knn(aod_df, ['AOD'])
df_imp_aod.to_csv("../data/imputed/misr-csn_aod-only_imputed.csv", index=False)

aod_prod = ['AOD_absorption', 'AOD_nonspherical', 'small_mode_AOD', 'medium_mode_AOD', 'large_mode_AOD']
df_imp_prod, prod_imp_ind = impute_knn(aod_df, aod_prod)
df_imp_prod.to_csv("../data/imputed/misr-csn_aod-prod_imputed.csv", index=False)

## Test model performance

In [ ]:
def run_base(df_nonNA, df_NA_imp, feature_list, res_base):
    """
    We train a base model with feature_list on df_nonNA (df with nonNA AOD vars) and 
    test performance on df_NA_imp (df with solely imputed NA AOD vars). Results
    are recorded in res_base and res_imp.
    """
    for sp in ['sulfate', 'nitrate', 'dust', 'EC', 'OC']:
        if sp == 'dust':
            df_new = df_nonNA.dropna(subset = ['Al', 'Si', 'Ca', 'Ti', 'Fe'] + feature_list)
            df_new['dust'] = 2.2*df_new['Al'] + 2.49*df_new['Si'] + 1.63*df_new['Ca'] + 1.94*df_new['Ti'] + 2.42*df_new['Fe']
            df_imp = df_NA_imp.dropna(subset = ['Al', 'Si', 'Ca', 'Ti', 'Fe'] + feature_list)
            df_imp['dust'] = 2.2*df_NA_imp['Al'] + 2.49*df_NA_imp['Si'] + 1.63*df_NA_imp['Ca'] + 1.94*df_NA_imp['Ti'] + 2.42*df_NA_imp['Fe']
        else:
            df_new = df_nonNA.dropna(subset = [sp] + feature_list)
            df_imp = df_NA_imp.dropna(subset = [sp] + feature_list)
        ## HANDLE OUTLIERS ##
        outlier_cutoff = 3 * np.percentile(df_new[sp], 75)
        imp_outlier_cutoff = 3 * np.percentile(df_imp[sp], 75)
        num_non, imp_num_non = df_new.shape[0], df_imp.shape[0]
        df_new = df_new[df_new[sp] <= outlier_cutoff]
        df_imp = df_imp[df_imp[sp] <= imp_outlier_cutoff]
        print('Outliers removed: ', num_non-df_new.shape[0], imp_num_non-df_imp.shape[0])
        print('Cutoff: ', outlier_cutoff, imp_outlier_cutoff)
        ####
        y = np.log(1 + df_new[sp])
        y_imp = np.log(1 + df_imp[sp])
        # y = df_new[sp]
        # y_imp = df_imp[sp]
        X_train, X_test, X_imp, y_train, y_test = transform(df_new[feature_list], None, df_imp[feature_list], y, \
            one_hot_cols = ['POC', 'fire_dist_25', 'ecosys'], split=True)
        mod = RandomForestRegressor()
        mod.fit(X_train, y_train)
        preds = mod.predict(X_test)
        preds_imp = mod.predict(X_imp)
        mod2 = xgb.XGBRegressor(objective="reg:squarederror")
        mod2.fit(X_train, y_train)
        preds2 = mod2.predict(X_test)
        preds2_imp = mod2.predict(X_imp)

        # Performance of base model on imputed data
        rmse_rf, r2_rf = get_perf(y_imp, preds_imp)
        # plot_fitted(y_imp, preds_imp, "Fitted vs True: " + sp + " (RF)", "../img/imputed/" + sp + "-fitted_rf.png")
        rmse_xg, r2_xg = get_perf(y_imp, preds2_imp)
        # plot_fitted(y_imp, preds2_imp, "Fitted vs True: " + sp + " (XGB)", "../img/imputed/" + sp + "-fitted_xgb.png")
        
        res_base = pd.concat([res_base, pd.DataFrame({'species': [sp, sp], 'r2':[r2_rf, r2_xg], 'rmse': [rmse_rf, rmse_xg], \
            'imp_method': ['base', 'base'], 'model': ['rf', 'xgb'], 'test_set': ['imp', 'imp']})])
        
        # Performance of base model on its own test set
        rmse_rf, r2_rf = get_perf(y_test, preds)
        # /img/sulfate/ft-imp/
        plot_fitted(y_test, preds, "Fitted vs True: " + sp + " (RF)", "../img/" + sp + "/fitted/" + sp + "-fitted_rf.png")
        rmse_xg, r2_xg = get_perf(y_test, preds2)
        # plot_fitted(y_test, preds2, "Fitted vs True: " + sp + " (XGB)", "../img/imputed/" + sp + "-fitted_xgb.png")

        plot_imp(mod, MODEL_FEATS['aod_only'], "Ft Importance for " + sp + " [" + 'aod_only' + "]", "".join(["../img/", sp, "/ft-imp/", sp, "-", "aod_only", "-ft_imp_rf.png"]))

        res_base = pd.concat([res_base, pd.DataFrame({'species': [sp, sp], 'r2':[r2_rf, r2_xg], 'rmse': [rmse_rf, rmse_xg], \
            'imp_method': ['base', 'base'], 'model': ['rf', 'xgb'], 'test_set': ['base', 'base']})])
        
    return res_base

In [ ]:
def run_imp(df_imp, feature_list, res_imp, exp_type):
    """
    Train model with feature_list on df_imp (complete df 
    including imputed data). Record test set results in res_imp.
    """
    for sp in ['sulfate', 'nitrate', 'dust', 'EC', 'OC']:
        if sp == 'dust':
            df_new = df_imp.dropna(subset = ['Al', 'Si', 'Ca', 'Ti', 'Fe'] + feature_list)
            df_new['dust'] = 2.2*df_new['Al'] + 2.49*df_new['Si'] + 1.63*df_new['Ca'] + 1.94*df_new['Ti'] + 2.42*df_new['Fe']
        else:
            df_new = df_imp.dropna(subset = [sp] + feature_list)
        ## HANDLE OUTLIERS ##
        outlier_cutoff = 3 * np.percentile(df_new[sp], 75)
        num_non = df_new.shape[0]
        df_new = df_new[df_new[sp] <= outlier_cutoff]
        print("Outliers removed: ", num_non-df_new.shape[0])
        print("Cutoff: ", outlier_cutoff)
        ####
        y = np.log(1 + df_new[sp])    
        # y = df_new[sp]
        X_train, X_test, _, y_train, y_test = transform(df_new[feature_list], None, None, y, \
            one_hot_cols = ['POC', 'fire_dist_25', 'ecosys'], split=True, exp_type=exp_type)
        mod = RandomForestRegressor()
        mod.fit(X_train, y_train)
        preds = mod.predict(X_test)
        mod2 = xgb.XGBRegressor(objective="reg:squarederror")
        mod2.fit(X_train, y_train)
        preds2 = mod2.predict(X_test)

        rmse_rf, r2_rf = get_perf(y_test, preds)
        rmse_xg, r2_xg = get_perf(y_test, preds2)
        res_imp = pd.concat([res_imp, pd.DataFrame({'species': [sp, sp], 'r2':[r2_rf, r2_xg], 'rmse': [rmse_rf, rmse_xg], \
            'imp_method': ['imp', 'imp'], 'model': ['rf', 'xgb'], 'test_set': ['imp', 'imp']})])    
        # plot_fitted(y_test, preds, " ".join(["Fitted vs True:", sp, "(RF)"]), "".join(["../img/imputed/wo-out/fitted/", sp, "-", exp_type, "-fitted_rf.png"]))
        # plot_imp(mod, MODEL_FEATS[exp_type], "Ft Importance for " + sp + " [" + exp_type + "]", "".join(["../img/imputed/wo-out/ft-imp/", sp, "-", exp_type, "-ft_imp_rf.png"]))
    return res_imp

### AOD Only

In [ ]:
# Define features for experiment
exp_type = 'aod_only'
feature_list = subset_features+ADD_FEATS[exp_type]

In [ ]:
### BASE MODEL ###
results_base_aod = pd.DataFrame({'species': [], 'r2':[], 'rmse': [], 'model': [], 'imp_method': [], 'test_set': []})
results_base_aod = run_base(aod_df.dropna(subset=ADD_FEATS[exp_type]), df_imp_aod.iloc[list(set(aod_imp_ind))], feature_list, results_base_aod)
results_base_aod.to_csv("results/results_base_aod.csv", index=False)

In [ ]:
### IMPUTED MODEL ###
results_imp_aod = pd.DataFrame({'species': [], 'r2':[], 'rmse': [], 'model': [], 'imp_method': [], 'test_set': []})
results_imp_aod = run_imp(df_imp_aod, feature_list, results_imp_aod, exp_type)
results_imp_aod.to_csv("results/results_imp_aod.csv", index=False)